In [1]:
import docx
import os
import pandas as pd
import io
from PIL import Image
import pytesseract
from wand.image import Image as wi
import fnmatch
import re
import shutil
from sklearn.externals import joblib

pytesseract.pytesseract.tesseract_cmd = r"C:\Users\caio.santos\AppData\Local\Tesseract-OCR\tesseract.exe"

pdfpath = r"C:\Users\caio.santos\Desktop\test constr"
docspath = r"C:\Users\caio.santos\Desktop\teste\docs"


def get_files_fromdirectory(in_files, out_files):
    for root, dirs, files in os.walk(in_files):  # replace the . with your starting directory
        for file in files:
            path_file = os.path.join(root, file)
            shutil.copy2(path_file, out_files)  # change you destination dir


def clause_split(data):
    docnum = []
    clausula = []
    text = []
    size = []
    

    rex = re.compile(r"cl[á,a]usula", flags=re.I)
    #rex = re.compile(r"(cl[á,a]usula)|(par[á,a]grafo\s\d+?º)|(\w+?:\d+?\.\d+?\.)", flags=re.I)
    ndoc = 0

    if type(data) == str:
        cl = 0
        ndoc += 1
        
        for string in re.split(rex, data):
            cl += 1
            docnum.append(ndoc)
            clausula.append(cl - 1)
            text.append(string)
            size.append(len(string))
            
            
    else:
        for d in data:
            ndoc += 1
            cl = 0
            T = model_type.predict(data)
            for string in re.split(rex, d):
                cl += 1
                docnum.append(ndoc)
                clausula.append(cl - 1)
                text.append(string)
                size.append(len(string))
                
                
    data = {'Doc': docnum, 'clausula': clausula, 'Text': text, 'Size': size}
    df = pd.DataFrame(data=data)
    df = df[df['Size']>120]
    df = df[df['Size']<15000]
    return df


def pdf2docx_convert(pdf_path, out):
    doc = docx.Document()
    text = get_pdf_text(pdf_path)
    doc.add_paragraph(text)
    doc.save(out + '.docx')


def get_pdf_text(pdf_path):
    pdf = wi(filename=pdf_path, resolution=300)
    pdfImg = pdf.convert('jpeg')
    imgBlobs = []
    text = []

    for img in pdfImg.sequence:
        page = wi(image=img)
        imgBlobs.append(page.make_blob('jpeg'))

    for imgBlob in imgBlobs:
        im = Image.open(io.BytesIO(imgBlob))
        tx = pytesseract.image_to_string(im, lang='por')
        text.append(tx)

    return text


def trainprepare(path):
    os.chdir(path)

    ndoc = 0
    docnum = []
    clausula = []
    text = []
    rex = re.compile(r"cl[á,a]usula", flags=re.I)
    for d in fnmatch.filter(os.listdir(), '*.docx'):
        doc = docx.Document(d)
        ndoc += 1
        cl = 0
        stringx = []
        for p in doc.paragraphs:
            stringx.append(p.text)

        for string in re.split(rex, ''.join(stringx)):
            cl += 1
            docnum.append(ndoc)
            clausula.append(cl - 1)
            text.append(string)

    data = {'Doc': docnum, 'clausula': clausula, 'Text': text}
    df = pd.DataFrame(data=data)
    return df


def trainprepare_DF_series(data):
    docnum = []
    clausula = []
    text = []

    #rex = re.compile(r"cl[á,a]usula", flags=re.I)
    rex = re.compile(r"(par[á,a]grafo\s\d+?º)|(\w+?:\d+?\.\d+?\.)")
    ndoc = 0
    for d in data:
        ndoc += 1
        cl = 0

        for string in re.split(rex, d):
            cl += 1
            docnum.append(ndoc)
            clausula.append(cl - 1)
            text.append(string)

    data = {'Doc': docnum, 'clausula': clausula, 'Text': text}
    df = pd.DataFrame(data=data)
    return df


def get_patter(df):
    text = df[0]
    cls = df[1]
    
    if cls == 'OBJ':
        string = []
        rex  = re.compile(r"XXXGGG?")
        rex2 = re.compile(r"[o,O]bjeto..+\.?\n?")
        for i in rex.findall(text):
            string.append(i)
        for i in rex2.findall(text):
            string.append(i)
        strin = '; '.join(string)

        return strin
    
    elif cls == 'VALOR':
        string = []
        rex  = re.compile(r"R\$\s?\d.+?\s")
        rex2 = re.compile(r"\d+?\%.+\.?")
        
        for i in rex.findall(text):
            string.append(i)
        for i in rex2.findall(text):
            string.append(i)
            
        strin = '; '.join(string)

        return strin
    
    elif cls == 'MULTA':
        string = []
        rex = re.compile(r"\d+?\%.+\.?\n?")
        rex2 = re.compile(r"R\$\s?\d.+?\s")


        for i in rex.findall(text):
            string.append(i)
            string.append('; ')
            
        for i in rex2.findall(text):
            string.append(i)
            string.append('; ')
        strin = ''.join(string)
        return strin

        
    elif cls == 'PRAZO':
        string = []
        rex = re.compile(r"\d\d\s?\(.+\)\s?\w+")
        rex2 = re.compile(r"\d\d\sde\s\w+\sde\s\d\d\d\d")
        rex3 = re.compile(r"\d\d\/\d\d\/\d\d\d\d")

        for i in rex.findall(text):
            string.append(i)
            string.append('; ')

        for i in rex2.findall(text):
            string.append(i)
            string.append('; ')

        for i in rex3.findall(text):
            string.append(i)
            string.append('; ')

        strin = ''.join(string)
        return strin

    else:
        return "nothing"


def contract_reader_from_file(file):

    model_clause = joblib.load('Clause_classifier')
    
    if file.lower().endswith(('.pdf', '.PDF')):
        string = get_pdf_text(file)
        text = ''.join(string)


    elif file.endswith('.docx'):
        doc = docx.Document(file)
        string = []
        for p in doc.paragraphs:
            string.append(p.text)
        text = ''.join(string)

    df = clause_split(text)
    cl_class = model_clause.predict(df['Text'])

    df['Cl class'] = cl_class

    df['Info'] = df[['Text', 'Cl class']].apply(get_patter, axis=1)
    return df


def contract_reader(data):
    model_type = joblib.load('Master_aditiv_classifier')
    model_clause = joblib.load('Clause_classifier')
           
    df = clause_split(data)
    cl_class = model_clause.predict(df['Text'])
    df['Cl class'] = cl_class
    df['Info'] = df[['Text', 'Cl class']].apply(get_patter, axis=1)
    return df

c:\users\caio.santos\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
path = r"C:\Users\caio.santos\Desktop\Set IFRS 16 contracts.xlsx"

In [7]:
df = pd.read_excel(path, sheet_name=1)

In [8]:
text = df['Text']

In [12]:
contract_reader(text[19])

,Doc,clausula,Text,Size,Cl class,Info
0,1,0,| sérgio Mártins\n\nATACADÃO\n\nVia Constavds ...,2493,SUMMARY,nothing
1,1,1,s e condições do Contrato para o novo período....,250,OUTROS,nothing
3,1,3,PRIMEIRA — DO PRAZO\n1.1. As partes concordam...,186,PRAZO,60 (sessenta) meses; 01/10/2017; 30/09/2022;
4,1,4,SEGUNDA — DO ALUGUEL\n2.1. O valor do aluguel...,802,VALOR,"R$78.538,70\n"
5,1,5,"TERCEIRA — DA GARANTIA ,\n\n3.1. A LOCATÁRIA ...",397,VALOR,
7,1,7,s.\n\n3.1.1. A FIADORA renuncia aos benefícios...,378,DIREITOS E DEVERES,nothing
8,1,8,QUARTA — DA DENOMINAÇÃO DA LOCATÁRIA/NOME FAN...,320,DIREITOS E DEVERES,nothing
9,1,9,"Primeira, Parágrafo\nPrimeiro do Contrato, qu...",242,VALOR,
10,1,10,PRIMEIRA — OBJETO DO CONTRATO\nE\n\nA LOCADOR...,1343,OBJ,objeto deste contrato destinar-se-á única e ex...
11,1,11,"\nnona. Todavia, o eventual consentimento para...",268,DIREITOS E DEVERES,nothing
